In [ ]:
try:
    import sys
    sys.path.append("/Users/adameshel/Documents/Python_scripts/my_functions/")
    from helper_functions import restartkernel
    restartkernel()
except:
    print('Get the function restartkernel from helper_functions')

In [1]:
import numpy as np
import pandas as pd
import glob
from collections import Counter
import sys
# sys.path.append("/Users/adameshel/Documents/Python_scripts/my_functions/") 
# from helper_functions import split_at

In [2]:
# path
path = '/Users/adameshel/Documents/Stuff/Basketball/Playoffs/2021/'

In [3]:
# real results

results_1 = ['PHI_5', 'ATL_5', 'MIL_4', 'BKN_5', 'UTA_5', 'DEN_6', 'PHX_6', 'LAC_7']

results_2 = ['ATL_7', 'MIL_7', 'PHX_4', 'LAC_6']
scorers_2 = ['CHPA', 'JOEM', 'KEDU', 'DOMI']

results_3 = ['PHX_6', 'MIL_6']
scorers_3 = ['PAGE','KHMI']
assisters_3 = ['CHPA','JRHO']
rebounders_3 = ['DEAY','CLCA']

results_4 = ['MIL_6']
scorers_4 = ['GIAN']
assisters_4 = ['JRHO']
rebounders_4 = ['GIAN']

standings_d = {
    'LAL':1.100000000,
    'NYK':1.200000000,
    'WAS':1.300000000,
    'POR':1.400000000,
    'MIL':3.75,
    'BKN':4.50,
    'BOS':5.00,
    'MIA':5.00,
    'PHI':6.50,
    'TOR':26.00,
    'CHI':36.00,
    'ATL':51.00,
    'CLE':91.00,
    'CHA':176.00,
    'PHX':2.25,
    'GSW':5.50,
    'MEM':7.00,
    'DAL':10.00,
    'DEN':11.00,
    'UTA':11.00,
    'LAC':23.00,
    'MIN':26.00,
    'SAS':251.00,
    'NOR':251.00
}

round_standings_d = {}
for k in list(standings_d.keys()):
    round_standings_d[k] = round(standings_d[k] ** 0.5, 2)


## From here on DO NOT change anything

In [10]:
late_penalty = 3
round_team_min = round_standings_d[min(round_standings_d, key=round_standings_d.get)]

def split_at(s, c, n):
    '''Function for splitting strings (first arg). The last argument is the position of the
        selected seperator (second arg) in the string.'''
    words = s.split(c)
    return c.join(words[:n]), c.join(words[n:])

def calc_scores(Playoff_round):
    if Playoff_round==1: 
        print('ROUND 1:\n\n')
        ROUND = 'first_round' + '/'
        TEAM = 3
        SCORE = 3
        results = results_1
        champ_score = 0
        print('POINTS\nTeam: %s, Score: %s\n' %(TEAM,SCORE))
        

    elif Playoff_round==2:
        print('ROUND 2:\n\n')
        ROUND = 'second_round' + '/'
        TEAM = 3.5
        SCORE = 4
        BEST_SCORER = 2
        results = results_2
        scorers = scorers_2
        champ_score = 0
        print('POINTS\nTeam: %s,\
        Score: %s, Scorer:%s\n' %(TEAM,SCORE,BEST_SCORER))

    elif Playoff_round==3:
        print('ROUND 3:\n\n')
        ROUND = 'third_round' + '/'
        TEAM = 5
        SCORE = 4
        BEST_SCORER = 3
        BEST_ASSISTER = 3
        BEST_REBOUNDER = 3
        results = results_3
        scorers = scorers_3
        assisters = assisters_3
        rebounders = rebounders_3
        champ_score = 0
        print('POINTS\nTeam: %s, Score: %s,\
        Scorer: %s, Assister: %s, Rebounder: %s \n' %(TEAM,
                                                      SCORE,
                                                     BEST_SCORER,
                                                     BEST_ASSISTER,
                                                     BEST_REBOUNDER))

    elif Playoff_round==4: 
        print('FINALS:\n\n')
        ROUND = 'finals' + '/'
        TEAM = 6
        SCORE = 5
        BEST_SCORER = 3
        BEST_ASSISTER = 3
        BEST_REBOUNDER = 3
        results = results_4
        scorers = scorers_4
        assisters = assisters_4
        rebounders = rebounders_4
        print('POINTS\nTeam: %s, Score: %s,\
        Scorer: %s, Assister: %s, Rebounder: %s \n' %(TEAM,
                                                     SCORE,
                                                     BEST_SCORER,
                                                     BEST_ASSISTER,
                                                     BEST_REBOUNDER))
        
        first_files = sorted(glob.glob(path + 'first_round/' + '*.txt'))
        list_of_names = []
        list_of_scores = []
        for i, file in enumerate(first_files):
            champ_score = 0
            f = open(file, "a+")
            f.write("\r\n")
            f.close()
            f = open(file)
            for j,line in enumerate(f):
                line = line.upper()
                if j==0:
                    name = line[:-1]
                    print(name)
                    while ' ' in name[-1]:
                        name = name[:-1]
                    continue
                if 'CHAMP' in line:
                    print(line)
                    champ_bet = split_at(line,',',1)[1]
                    champ_bet = champ_bet[0:3]
                    if champ_bet in results_4[0]:
                        champ_score = standings_d[champ_bet]
                    list_of_scores.append(champ_score)
                    list_of_names.append(name)
                    f.close()
                    break
                else:
                    continue
                
        d_champ = dict(zip(list_of_names, list_of_scores))
                    
        
    files = sorted(glob.glob(path + ROUND + '*.txt'))
    list_of_names = []
    list_of_scores = []
    for i, file in enumerate(files):
        my_team_result = 0
        my_score_result = 0
        my_best_scorer_result = 0
        my_best_assister_result = 0
        my_best_rebounder_result = 0
        my_penalty_result = 0
        f = open(file, "a+")
        f.write("\r\n")
        f.close()
        f = open(file)
        for j,line in enumerate(f):
            line = line.upper()
            if j==0:
                name = line[:-1]
                while ' ' in name[-1]:
                    name = name[:-1]
                list_of_names.append(name)
                continue
            if '%' in line:
                continue
            if ',' in line and '_' in line:
                team_bet = split_at(split_at(line,',',1)[1],'_',1)[0]
                opponent1 = split_at(split_at(line,',',1)[0],'_',1)[0]
                opponent2 = split_at(split_at(line,',',1)[0],'_',1)[1]
                score_bet = split_at(split_at(line,',',1)[1],'_',1)[1][0]
                for result in results:
                    norm_base_for_match = min(round_standings_d[opponent1],
                                              round_standings_d[opponent2])
                    if team_bet in result:       
                        my_team_result += round(
                            TEAM * (round_standings_d[split_at(result,'_',1)[0]] / norm_base_for_match),
                            2
                        )
                        print('Ratio rewarded for the series ' + str(split_at(line,',',1)[0]) + ':')
                        print(str(TEAM) + ' * ' +\
                              str(round(round_standings_d[split_at(result,'_',1)[0]] / norm_base_for_match,2)))
                        # check if score is correct
                        if score_bet in result:
                            my_score_result += SCORE
            if 'SCORERS:' in line:
                players = split_at(line,':',1)[1][:-1].split(',')
                for player in players:
                    if player in scorers:
                        my_best_scorer_result += BEST_SCORER
            if 'ASSISTERS:' in line:
                players = split_at(line,':',1)[1][:-1].split(',')
                for player in players:
                    if player in assisters:
                        my_best_assister_result += BEST_ASSISTER
            if 'REBOUNDERS:' in line:
                players = split_at(line,':',1)[1][:-1].split(',')
                for player in players:
                    if player in rebounders:
                        my_best_rebounder_result += BEST_REBOUNDER 
            if 'PENALTY:' in line:
                penalty = int(split_at(line,':',1)[1][:-1].split(',')[0]) * late_penalty
                my_penalty_result += penalty

            if Playoff_round==4: 
                champ_score = d_champ[name]
        temp_total = round(my_team_result +\
                     my_score_result +\
                     my_best_scorer_result +\
                     my_best_assister_result +\
                     my_best_rebounder_result +\
                     champ_score -\
                     my_penalty_result,
                           2
                    )
        list_of_scores.append(temp_total)

        f.close()

        if Playoff_round==1: 
            if my_penalty_result==0:
                print(name + '\n','Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result), 
                      'Total for this round: ' + str(temp_total),
                      '\n')
            else:
                print(name + '\n','Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result),
                      'Penalty: ' + str(my_penalty_result),
                      'Total for this round: ' + str(temp_total),
                      '\n')

        elif Playoff_round==2:
            if my_penalty_result==0:
                print(name + '\n','Team: ' + str(my_team_result),
                  'Score: ' + str(my_score_result),
                  'Scorer: ' + str(my_best_scorer_result),
                  'Total for this round: ' + str(temp_total),
                  '\n')
            else:
                print(name + '\n','Team: ' + str(my_team_result),
                  'Score: ' + str(my_score_result),
                  'Scorer: ' + str(my_best_scorer_result),
                  'Penalty: ' + str(my_penalty_result),
                  'Total for this round: ' + str(temp_total),
                  '\n')
        elif Playoff_round==3:
            if my_penalty_result==0:
                print(name + '\n','Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result),
                      'Scorer: ' + str(my_best_scorer_result),
                      'Assister: ' + str(my_best_assister_result),
                      'Rebounder: ' + str(my_best_rebounder_result),
                      'Total for this round: ' + str(temp_total),
                      '\n')
            else:
                print(name + '\n','Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result),
                      'Scorer: ' + str(my_best_scorer_result),
                      'Assister: ' + str(my_best_assister_result),
                      'Rebounder: ' + str(my_best_rebounder_result),
                      'Penalty: ' + str(my_penalty_result),
                      'Total for this round: ' + str(temp_total),
                      '\n')
        else:
            if my_penalty_result==0:
                print(name + '\n','Team: ' + str(my_team_result),
                      'Score: ' + str(my_score_result),
                      'Scorer: ' + str(my_best_scorer_result),
                      'Assister: ' + str(my_best_assister_result),
                      'Rebounder: ' + str(my_best_rebounder_result),
                      'Champion: ' + str(champ_score),
                      'Total for this round: ' + str(temp_total),
                      '\n')
            else:
                print(name + '\n','Team: ' + str(my_team_result),
                  'Score: ' + str(my_score_result),
                  'Scorer: ' + str(my_best_scorer_result),
                  'Assister: ' + str(my_best_assister_result),
                  'Rebounder: ' + str(my_best_rebounder_result),
                  'Champion: ' + str(champ_score),
                  'Penalty: ' + str(my_penalty_result),
                  'Total for this round: ' + str(temp_total),
                  '\n')
            
    print('\n\n\n')
    d = dict(zip(list_of_names, list_of_scores))
    return d

def calc_total(round_number):
    if round_number == 1:
        d_comb = calc_scores(Playoff_round=round_number)
    elif round_number == 2:
        d_1 = calc_scores(Playoff_round=round_number-1)
        d_comb = dict(Counter(d_1)+Counter(calc_scores(Playoff_round=round_number)))
    elif round_number == 3:
        d_1 = calc_scores(Playoff_round=round_number-2)
        d_2 = dict(Counter(d_1)+Counter(calc_scores(Playoff_round=round_number-1)))
        d_comb = dict(Counter(d_2)+Counter(calc_scores(Playoff_round=round_number)))
    elif round_number == 4:
        d_1 = calc_scores(Playoff_round=round_number-3)
        d_2 = dict(Counter(d_1)+Counter(calc_scores(Playoff_round=round_number-2)))
        d_3 = dict(Counter(d_2)+Counter(calc_scores(Playoff_round=round_number-1)))
        d_comb = dict(Counter(d_3)+Counter(calc_scores(Playoff_round=round_number)))
    else:
        print('There is a problem with the scores dictionaries')
    print('TOTALS:\n\n')
    sort_orders = sorted(d_comb.items(), key=lambda x: x[1], reverse=True)
    pos_count = 0
    for i in sort_orders:
        pos_count+=1
        print(str(pos_count) + '. '+ str(i[0]))
        print('    ' + str(i[1]))
        print('\n')
#     for key in d_comb.keys():
#         print(key, '\t')
#         print(d_comb[key], '\n')

In [11]:
calc_total(round_number=4)

ROUND 1:


POINTS
Team: 3, Score: 3

Ratio rewarded for the series UTA_MEM:
3 * 1.25
Ratio rewarded for the series LAC_DAL:
3 * 1.52
Ratio rewarded for the series DEN_POR:
3 * 2.81
Ratio rewarded for the series PHI_WAS:
3 * 2.24
Ratio rewarded for the series NYK_ATL:
3 * 6.49
Ratio rewarded for the series BKN_BOS:
3 * 1.0
ADAM_ESHEL
 Team: 45.94 Score: 9 Total for this round: 54.94 

Ratio rewarded for the series UTA_MEM:
3 * 1.25
Ratio rewarded for the series LAC_DAL:
3 * 1.52
Ratio rewarded for the series DEN_POR:
3 * 2.81
Ratio rewarded for the series PHI_WAS:
3 * 2.24
Ratio rewarded for the series MIL_MIA:
3 * 1.0
Ratio rewarded for the series BKN_BOS:
3 * 1.0
ALON_KESTECHER
 Team: 29.47 Score: 9 Total for this round: 38.47 

Ratio rewarded for the series UTA_MEM:
3 * 1.25
Ratio rewarded for the series LAC_DAL:
3 * 1.52
Ratio rewarded for the series DEN_POR:
3 * 2.81
Ratio rewarded for the series PHI_WAS:
3 * 2.24
Ratio rewarded for the series NYK_ATL:
3 * 6.49
Ratio rewarded for t

# Print all bets for a specific round

In [4]:
first_files = sorted(glob.glob(path + 'finals/' + '*.txt'))
list_of_names = []
list_of_scores = []
for i, file in enumerate(first_files):
    print('\n')
    champ_score = 0
    f = open(file, "a+")
    f.write("\r\n")
    f.close()
    f = open(file)
    for j,line in enumerate(f):
        line = line.upper()
        print(line)



HORESH



PHX_MIL,PHX_5



SCORERS:DEBO

ASSISTERS:CHPA

REBOUNDERS:DEAY



YOTAM_ZAIT



PHX_MIL_MIL_6



SCORERS:DEBO

ASSITERS:CHPA

REBOUNDERS:BRLO



GILAD_SOKOLOVER



PHX_MIL,MIL_6



SCORERS:DEBO

ASSISTERS:CHPA

REBOUNDERS:DEAY



HAMEIROV



PHX_MIL,PHX_5



SCORERS:JRHO

ASSISTERS:CHPA

REBOUNDERS:BRLO



OFER_KLONER



ALON_KESTECHER



PHX_MIL,PHX_7



SCORERS:DEBO

ASSISTERS:CHPA

REBOUNDERS:DEAY



ROY_YANOVSKI



PHX_MIL,PHX_5



SCORERS:DEBO

ASSISTERS:JRHO

REBOUNDERS:DEAY



ROY_ESHEL



PHX_MIL,MIL_5



SCORERS:KHMI

ASSISTERS:CHPA

REBOUNDERS:DEAY



EDEN_ESHEL



PHX_MIL,PHX_7



SCORERS:DEBO

ASSISTERS:CHPA

REBOUNDERS:DEAY



YF_TZ



PHX_MIL,PHX_4



SCORERS:DEBO

ASSISTERS:JRHO

REBOUNDERS:PJTU



DOLF



PHX_MIL,PHX_5



SCORERS:KHMI

ASSISTERS:CHPA

REBOUNDERS:DEAY



AVIV_KLEIN



PHX_MIL,PHX_6



SCORERS:KHMI

ASSISTERS:CHPA

REBOUNDERS:DEAY



ADAM_ESHEL



PHX_MIL,MIL_7



SCORERS:CHPA

ASSISTERS:JRHO

REBOUNDERS:BRLO

